# Homework 2: training pipeline

This code will test your homework 2 solutions by using them in a complete ML pipeline. You should run this code in order to tune your model and save your model weights (which will also be uploaded as part of your solution)

In [1]:
# Download the training data from the homework2 folder:
# unzip using tar xzvvf nsynth_subset.tar.gz
# (this is a small subset of the "nsynth" dataset: https://magenta.tensorflow.org/datasets/nsynth)

In [2]:
import homework2

['./nsynth_subset/guitar_acoustic_021-101-050.wav', './nsynth_subset/guitar_acoustic_015-077-050.wav', './nsynth_subset/guitar_acoustic_014-054-075.wav', './nsynth_subset/vocal_synthetic_003-024-100.wav', './nsynth_subset/guitar_acoustic_021-085-100.wav', './nsynth_subset/vocal_synthetic_003-046-075.wav', './nsynth_subset/guitar_acoustic_010-066-127.wav', './nsynth_subset/guitar_electronic_022-083-100.wav', './nsynth_subset/guitar_acoustic_010-073-127.wav', './nsynth_subset/guitar_electronic_022-077-075.wav', './nsynth_subset/guitar_acoustic_010-083-127.wav', './nsynth_subset/vocal_acoustic_000-053-075.wav', './nsynth_subset/vocal_synthetic_003-053-100.wav', './nsynth_subset/guitar_acoustic_015-085-100.wav', './nsynth_subset/guitar_electronic_028-054-127.wav', './nsynth_subset/vocal_synthetic_003-068-075.wav', './nsynth_subset/guitar_acoustic_014-106-075.wav', './nsynth_subset/vocal_synthetic_003-049-025.wav', './nsynth_subset/vocal_synthetic_003-061-100.wav', './nsynth_subset/vocal_ac

### Install and Load Required Libraries  

In [3]:
# !pip install librosa
# !pip install torch
# !pip install glob
# !pip install numpy

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as nnF
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
import librosa
import random
import glob

In [5]:
BATCH_SIZE = 16
torch.use_deterministic_algorithms(True)

In [6]:
if not len(homework2.audio_paths):
    print("You probably need to set the dataroot folder correctly")

In [7]:
# Some helper functions. These are the same as what the autograder runs.

In [8]:
# Split dataset into train / valid / test
def split_data(waveforms, labels, train_ratio=0.7, valid_ratio=0.15):
    assert(train_ratio + valid_ratio < 1)
    test_ratio = 1 - (train_ratio + valid_ratio)
    N = len(waveforms)
    Ntrain = int(N * train_ratio)
    Nvalid = int(N * valid_ratio)
    Ntest = int(N * test_ratio)
    Wtrain = waveforms[:Ntrain]
    Wvalid = waveforms[Ntrain:Ntrain + Nvalid]
    Wtest = waveforms[Ntrain + Nvalid:]
    ytrain = labels[:Ntrain]
    yvalid = labels[Ntrain:Ntrain + Nvalid]
    ytest = labels[Ntrain + Nvalid:]
    return Wtrain,Wvalid,Wtest,ytrain,yvalid,ytest

In [9]:
def process_data(W, feature_function):
    return [feature_function(path) for path in W]

In [10]:
class InstrumentDataset(Dataset):
    def __init__(self, features, labels):
        self.features = features
        self.labels = labels
        
    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        features = self.features[idx]
        label = self.labels[idx]

        return features, torch.tensor(label, dtype=torch.long)

In [11]:
class Loaders():
    def __init__(self, waveforms, labels, feature_function, seed = 0):
        torch.manual_seed(seed)
        random.seed(seed)
        self.Wtrain, self.Wvalid, self.Wtest, self.ytrain, self.yvalid, self.ytest = split_data(waveforms, labels)
        
        self.Xtrain = process_data(self.Wtrain, feature_function)
        self.Xvalid = process_data(self.Wvalid, feature_function)
        self.Xtest = process_data(self.Wtest, feature_function)
        
        self.dataTrain = InstrumentDataset(self.Xtrain, self.ytrain)
        self.dataValid = InstrumentDataset(self.Xvalid, self.yvalid)
        self.dataTest = InstrumentDataset(self.Xtest, self.ytest)
        
        self.loaderTrain = DataLoader(self.dataTrain, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)
        self.loaderValid = DataLoader(self.dataValid, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)
        self.loaderTest = DataLoader(self.dataTest, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)

In [12]:
class Pipeline():
    def __init__(self, module, learning_rate, seed = 0):
        # These two lines will (mostly) make things deterministic.
        # You're welcome to modify them to try to get a better solution.
        torch.manual_seed(seed)
        random.seed(seed)

        self.device = torch.device("cpu") # Can change this if you have a GPU, but the autograder will use CPU
        self.criterion = nn.CrossEntropyLoss()
        
        self.model = module.to(self.device)
        self.optimizer = optim.Adam(self.model.parameters(), lr=learning_rate)

    def evaluate(self, loader, which = "valid"):
        self.model.eval()

        correct = 0
        total = 0

        with torch.no_grad():
            for inputs, labels in loader:
                inputs, labels = inputs.to(self.device), labels.to(self.device)

                outputs = self.model(inputs)
                #loss = criterion(outputs, labels) # validation loss

                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        acc = correct / total
        
        return acc
    
    def train(self, loaders,
          num_epochs=1, # Train for a single epoch by default
          model_path=None): # (Optionally) provide a path to save the best model
        val_acc = 0
        best_val_acc = 0
        for epoch in range(num_epochs):
            self.model.train()
            
            losses = []

            for inputs, labels in loaders.loaderTrain:
                inputs, labels = inputs.to(self.device), labels.to(self.device)

                self.optimizer.zero_grad()
                outputs = self.model(inputs)
                loss = self.criterion(outputs, labels)
                loss.backward()
                self.optimizer.step()
                losses.append(float(loss))
            
            self.model.eval()
            val_acc = self.evaluate(loaders.loaderValid)
            print("Epoch " + str(epoch) + ", loss = " + str(sum(losses)/len(losses)) +\
                  ", validation accuracy = " + str(val_acc))

            if val_acc > best_val_acc:
                best_val_acc = val_acc
                if (model_path):
                    torch.save(self.model.state_dict(), model_path)
        print("Final validation accuracy = " + str(val_acc) + ", best = " + str(best_val_acc))
        return val_acc, best_val_acc

    def load(self, path):
        self.model.load_state_dict(torch.load(path, weights_only=True))

In [13]:
# The function below is the basis of how the autograder tests your code. Try to understand this one.

In [14]:
def test(waveforms, labels, feature_func, classifier, learning_rate, path):
    print("Extracting features...")
    test_loaders = Loaders(waveforms, labels, feature_func)
    test_pipeline = Pipeline(classifier, learning_rate)
    
    # Note: the autograder will not run this line: it will just load your saved model (next line)
    acc, best_acc = test_pipeline.train(test_loaders, 20, path)
    
    test_pipeline.load(path)
    test_acc = test_pipeline.evaluate(test_loaders.loaderTest)
    print("Test accuracy = " + str(test_acc))

In [15]:
# 1. Paths, labels, waveforms

In [16]:
# Once you've written the corresponding code in homework2.py, print these out or visualize them if you want
homework2.waveforms
homework2.labels

[0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,


In [17]:
# 2. MFCC

In [18]:
"""test(homework2.waveforms,
     homework2.labels,
     homework2.extract_mfcc,
     homework2.MLPClassifier(),
     0.0001,
     "best_mlp_model.weights")"""

'test(homework2.waveforms,\n     homework2.labels,\n     homework2.extract_mfcc,\n     homework2.MLPClassifier(),\n     0.0001,\n     "best_mlp_model.weights")'

In [19]:
# 3. Spectrogram

In [20]:
def test2(waveforms, labels, feature_func, classifier, learning_rate, path):
    print("Extracting features...")
    test_loaders = Loaders(waveforms, labels, feature_func)
    test_pipeline = Pipeline(classifier, learning_rate)
    
    # Note: the autograder will not run this line: it will just load your saved model (next line)
    acc, best_acc = test_pipeline.train(test_loaders, 10, path)
    
    test_pipeline.load(path)
    test_acc = test_pipeline.evaluate(test_loaders.loaderTest)
    print("Test accuracy = " + str(test_acc))

In [30]:
test2(homework2.waveforms,
     homework2.labels,
     homework2.extract_spec,
     homework2.SimpleCNN(),
     0.002,
     "best_spec_model.weights")

Extracting features...
Epoch 0, loss = 1.3968323138025072, validation accuracy = 0.4796747967479675
Epoch 1, loss = 1.23247605893347, validation accuracy = 0.4796747967479675
Epoch 2, loss = 1.0970322291056316, validation accuracy = 0.5203252032520326
Epoch 3, loss = 0.98141258292728, validation accuracy = 0.4796747967479675
Epoch 4, loss = 0.8766135291920768, validation accuracy = 0.4796747967479675
Epoch 5, loss = 0.8110062546200223, validation accuracy = 0.4796747967479675
Epoch 6, loss = 0.7733208719227049, validation accuracy = 0.5203252032520326
Epoch 7, loss = 0.7537201179398431, validation accuracy = 0.5203252032520326
Epoch 8, loss = 0.7420068879922231, validation accuracy = 0.5203252032520326
Epoch 9, loss = 0.733678874042299, validation accuracy = 0.5203252032520326
Final validation accuracy = 0.5203252032520326, best = 0.5203252032520326
Test accuracy = 0.47580645161290325


In [22]:
# 4. Mel-spectrogram

In [23]:
test(homework2.waveforms,
     homework2.labels,
     homework2.extract_mel,
     homework2.SimpleCNN(),
     0.0001,
     "best_mel_model.weights")

Extracting features...
Epoch 0, loss = 1.113267946574423, validation accuracy = 0.6585365853658537
Epoch 1, loss = 0.8253344297409058, validation accuracy = 0.8211382113821138
Epoch 2, loss = 0.6976935383346345, validation accuracy = 0.8536585365853658
Epoch 3, loss = 0.6105117425322533, validation accuracy = 0.8780487804878049
Epoch 4, loss = 0.5420914474460814, validation accuracy = 0.9105691056910569
Epoch 5, loss = 0.4886576483647029, validation accuracy = 0.9105691056910569
Epoch 6, loss = 0.4437347853349315, validation accuracy = 0.9105691056910569
Epoch 7, loss = 0.40496305872996646, validation accuracy = 0.9186991869918699
Epoch 8, loss = 0.3709080782201555, validation accuracy = 0.9349593495934959
Epoch 9, loss = 0.340556252333853, validation accuracy = 0.9512195121951219
Epoch 10, loss = 0.31330892195304233, validation accuracy = 0.9512195121951219
Epoch 11, loss = 0.2888600337836478, validation accuracy = 0.959349593495935
Epoch 12, loss = 0.267014498098029, validation accur

In [24]:
# 5. Constant-Q

In [25]:
test(homework2.waveforms,
     homework2.labels,
     homework2.extract_q,
     homework2.SimpleCNN(),
     0.0001,
     "best_q_model.weights")

Extracting features...
Epoch 0, loss = 1.4339646134111617, validation accuracy = 0.45528455284552843
Epoch 1, loss = 1.2188792874415715, validation accuracy = 0.7073170731707317
Epoch 2, loss = 1.0992902633216646, validation accuracy = 0.7317073170731707
Epoch 3, loss = 1.0005565070443683, validation accuracy = 0.7479674796747967
Epoch 4, loss = 0.914331368274159, validation accuracy = 0.8130081300813008
Epoch 5, loss = 0.8353197044796414, validation accuracy = 0.8292682926829268
Epoch 6, loss = 0.7629594951868057, validation accuracy = 0.8292682926829268
Epoch 7, loss = 0.6974902682834201, validation accuracy = 0.8617886178861789
Epoch 8, loss = 0.6374171202381452, validation accuracy = 0.8699186991869918
Epoch 9, loss = 0.5826657257146306, validation accuracy = 0.9024390243902439
Epoch 10, loss = 0.5337613365716405, validation accuracy = 0.9105691056910569
Epoch 11, loss = 0.4894355411330859, validation accuracy = 0.9186991869918699
Epoch 12, loss = 0.44921168436606723, validation ac

In [26]:
# 6. Pitch shift

In [27]:
test(homework2.augmented_waveforms,
     homework2.augmented_labels,
     homework2.extract_q,
     homework2.SimpleCNN(),
     0.0001,
     "best_augmented_model.weights")

Extracting features...
Epoch 0, loss = 1.2949702651412398, validation accuracy = 0.7615176151761518
Epoch 1, loss = 0.9817094858045932, validation accuracy = 0.8373983739837398
Epoch 2, loss = 0.7885123433338271, validation accuracy = 0.8861788617886179
Epoch 3, loss = 0.6466331217024062, validation accuracy = 0.9105691056910569
Epoch 4, loss = 0.538546639184157, validation accuracy = 0.9376693766937669
Epoch 5, loss = 0.45766130269125654, validation accuracy = 0.94579945799458
Epoch 6, loss = 0.39613703003636114, validation accuracy = 0.948509485094851
Epoch 7, loss = 0.34748075529932976, validation accuracy = 0.9539295392953929
Epoch 8, loss = 0.3077817623400026, validation accuracy = 0.9539295392953929
Epoch 9, loss = 0.27513409468034905, validation accuracy = 0.967479674796748
Epoch 10, loss = 0.24771138515185426, validation accuracy = 0.9701897018970189
Epoch 11, loss = 0.22494863222042719, validation accuracy = 0.9728997289972899
Epoch 12, loss = 0.20551166493721582, validation a

In [28]:
# 7. Extend your model to handle four classes and creatively improve its performance

In [29]:
test(homework2.waveforms,
     homework2.labels_7,
     homework2.feature_func_7,
     homework2.model_7,
     0.005,
     "best_model_7.weights")



Extracting features...
Epoch 0, loss = 1.7717977036308084, validation accuracy = 0.8617886178861789
Epoch 1, loss = 0.37997635919600725, validation accuracy = 0.8536585365853658
Epoch 2, loss = 0.2903559967978961, validation accuracy = 0.8292682926829268
Epoch 3, loss = 0.26327795561196077, validation accuracy = 0.8292682926829268
Epoch 4, loss = 0.2505117817264464, validation accuracy = 0.8455284552845529
Epoch 5, loss = 0.24393620124707618, validation accuracy = 0.8455284552845529
Epoch 6, loss = 0.2374691921286285, validation accuracy = 0.8455284552845529
Epoch 7, loss = 0.2428266442536066, validation accuracy = 0.8780487804878049
Epoch 8, loss = 0.25501010028852356, validation accuracy = 0.8943089430894309
Epoch 9, loss = 0.2247639392864787, validation accuracy = 0.8699186991869918
Epoch 10, loss = 0.21159589013809132, validation accuracy = 0.8780487804878049
Epoch 11, loss = 0.20717468117881152, validation accuracy = 0.8861788617886179
Epoch 12, loss = 0.1927529831106464, validati